# Memoria dinámica en C

```{admonition} Objetivos
* Objetivo 1...
* Objetivo 2...
```

## 1. Introduccion

En los programas tipicos, la cantidad de memoria que usan es desconocida. Por ejemplo, cuando alguien emplea un editor de texto nunca sabe cuantas paginas va a escribir. Como el numero de paginas suele estar relacionada con elementos de memoria como matrices presuponer un numero maximo de paginas suele ser ineficiente, pues ello implicaria dos cosas:
1. Que se tenga gran cantidad de memoria ociosa si el usuario del editor no escribe en todas las paginas reservadas por la aplicacion.
2. Que el usuario se quede corto de paginas si escribe mucho, pues al tener el editor de texto el numero de paginas predefinido, es posible que el espacio en memoria no sea suficiente para lo que el usuario escribe.

Para dar administrar la memoria de manera mas eficiente, C permite introduce el concepto de **reserva dinamica de memoria** que no es otra cosa que el proceso de gestion de memoria en tiempo de ejecución segun las necesidades demandadas por la aplicacion. Gracias a esto, es posible que no sea necesario que el editor de texto conozca con antelacion el numero de paginas que debe escribir el usuario, pues gracias a la gestion dinamica de memoria, el editor puede ir creando nuevas paginas a medida que el usuario va demandandolo. En esta seccion se verá las funciones que son empleadas en C para hacer esto posible.

## 2. Mapa de memoria

Cuando un programa es cargado en memoria, este se organiza en diferentes regiones memoria conocidas como **segmentos** que se usan para diferentes propositos.

```{figure} ./local/img/CH_02-S05-fig1_3.png
---
name: fig1
---
Mapa de memoria (Memory Layout)
```

* **Text Segment**: Tambien conocido como **code segment**. Es el lugar en el que se encuentra el código ejecutable (o binario) asociado al programa. 
* **Data Segment**: Segmento donde se encuentran las variables y constantes asociadas al programa. Este se divide en:
  * **Initialized data segment**: Aqui se almacenan las variables globales y estaticas que son inicializadas antes de la ejecución del programa (inicialización explicita).
  * **Uninitialized data segment (BSS - Block Started by Symbol)**: Contiene tolas las variables globales y estadicas que no se inicializaron explicitamente.
* **Heap**: Segmento utilizado para la asignación de memoria durante la ejecución del programa. La asignación y liberación de memoria se hace mediante funciones como `malloc`, `calloc`, `realloc` y `free`. Es importante aclarar que esta es una región de caracter dinamico, pues su tamaño cambia a lo largo de la ejecución del programa.
* **Stack**: Región empleada para almacenar las variables locales. Esta región tambien es dinamica ya que su tamaño cambia de manera automatica conforme a medida que se usan funciones.

Existen algunas represetaciones en la cuales los segmentos de **Text Segment** y **Data Segment** se combinan como un segmento conocido como **Program Code** tal y como se muestra a continuación:

```{figure} ./local/img/CH_02-S05-fig2.png
---
name: fig2
---
Mapa de memoria (Memory Layout)
```

### Ejemplos

#### Ejemplo 1

Simule el siguiente código y analice en como sería el mapa de memoria asociado al programa.

```{code-block} c
int x=5;
char msg[] = "Hello";

int main(int argc, const char* argv[]) {
  int v;
  float pi = 3.14159;
  printf("%d\n",x);
  printf("%f\n",pi);
  return 0;
}
```

La simulación se puede realizar a continuación:

<iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=%23include%20%3Cstdio.h%3E%0A%0Aint%20x%3D5%3B%0Achar%20msg%5B%5D%20%3D%20%22Hello%22%3B%0A%0Aint%20main%28int%20argc,%20const%20char*%20argv%5B%5D%29%20%7B%0A%20%20int%20v%3B%0A%20%20float%20pi%20%3D%203.14159%3B%0A%20%20printf%28%22%25d%5Cn%22,x%29%3B%0A%20%20printf%28%22%25f%5Cn%22,pi%29%3B%0A%20%20return%200%3B%0A%7D&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

El resultado de la simulación se muestra a continuación:

```{figure} ./local/img/CH_02-S05-fig3.png
---
name: fig3
---
Salida del programa.
```

Como se puede ver de la figura anterior, se resaltan los espacios de memoria y las variables que se encuentran en estos:

|Espacio de memoria |	Variables |
|----|----|
|**Text**|Instrucciones|
|**Data**|`x`, `msg`|
|**Stack**|`argc`, `argv`, `v`, `pi`|
|**Heap**|---|

En la siguiente figura se muestra mas claramente como se distribuyen las variables anteriores en el mapa de memoria:

```{figure} ./local/img/CH_02-S05-fig4.png
---
name: fig4
---
Asignacion de las variables en el espacio de memoria.
```

En la figura anterior, se puede ver que no hay variables en el **heap** pues no se hicieron asignaciones dinamicas de memoria el cual sera el proximo tema a tratar.

#### Ejemplo 2

Dado el siguiente código (tomado del documento **Memory & C** ([link](https://d1b10bmlvqabco.cloudfront.net/attach/j6fe5friemd22w/hzd1madqsie3ts/j7kw6i4tmqf8/61C_Note_1_Memory.pdf))) como seria el mapa de memoria asociado.

```{code-block} c
#include <stdio.h>
#include <stdlib.h>

int z = -1;

int main(int argc, char* argv) {
    int x = 3 * sizeof(int);
    char* s_static = "61C";
    char s_stack[4];
    s_stack[0] = '6';
    s_stack[1] = '1';
    s_stack[2] = 'B';
    s_stack[3] = '\0';
    int *heap_arr = (int *)malloc(x);
    heap_arr[0] = 1;
    heap_arr[1] = 2;
    heap_arr[2] = 4;
}
```

El programa se puede simular a continuación:

<iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstdlib.h%3E%0A%0Aint%20z%20%3D%20-1%3B%0A%0Aint%20main%28int%20argc,%20char*%20argv%29%20%7B%0A%20%20%20%20int%20x%20%3D%203%20*%20sizeof%28int%29%3B%0A%20%20%20%20char*%20s_static%20%3D%20%2261C%22%3B%0A%20%20%20%20char%20s_stack%5B4%5D%3B%0A%20%20%20%20s_stack%5B0%5D%20%3D%20'6'%3B%0A%20%20%20%20s_stack%5B1%5D%20%3D%20'1'%3B%0A%20%20%20%20s_stack%5B2%5D%20%3D%20'B'%3B%0A%20%20%20%20s_stack%5B3%5D%20%3D%20'%5C0'%3B%0A%20%20%20%20int%20*heap_arr%20%3D%20%28int%20*%29malloc%28x%29%3B%0A%20%20%20%20heap_arr%5B0%5D%20%3D%201%3B%0A%20%20%20%20heap_arr%5B1%5D%20%3D%202%3B%0A%20%20%20%20heap_arr%5B2%5D%20%3D%204%3B%0A%7D&codeDivHeight=400&codeDivWidth=350&cppShowMemAddrs=true&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

La siguiente figura muestra el resultado de la simulación:

```{figure} ./local/img/CH_02-S05-fig5.png
---
name: fig5
---
Asignacion de las variables en el espacio de memoria.
```

|Espacio de memoria |	Variables |
|----|----|
|**Text**|Instrucciones|
|**Data**|`z`|
|**Stack**|`argc`, `argv`, `x`, `s static`, `s_stack[]`, `heap_arr`|
|**Heap**|`heap arr[0]`, `heap arr[3]`, `heap arr[2]`|

## 3. Asignando memoria

Para tratar esta parte abordemos un problema tipico para ver las diferentes formas de solucion desde el punto de vista del manejo de memoria. Supongase que se desean ingresar un conjunto de datos asociados con la temperatura a lo largo del dia, el ingreso de estos datos se hace de manera manual leyendo el numero de registros manuales existentes en una planilla para luego ingresarlos al sistema. Hacer un programa que facilite esta tarea.

Existen diferentes maneras de asignar memoria.

### Forma 1

Declarar un arreglo estatico de tamaño fijo asumiento que el numero de registros a ingresar nunca va a ser mayor que el tamaño fijo previamente definido. Esta forma de declaración es **estatica** por que la reserva de memoria (arreglo) se hace en tiempo de compilación. A continuación se muestra el código:

```C
#include <stdio.h>

#define NUM_MAX_REG 100

int main() {
  float datos[NUM_MAX_REG]; // Se supone que no se van a ingresar mas de 100 registros
  float reg;
  int numReg;
  printf("Ingrese la cantidad de registros a leer: ");
  scanf("&d",numReg);
  if (numReg > NUM_MAX_REG) {
    printf("ERROR: se pierden %d datos\n", numReg - NUM_MAX_REG);
    numReg = NUM_MAX_REG;
  } 
  for(int i = 0; i < numReg; i++) {
    printf("Dato[%d]: ", i);
    scanf("%f", &reg);    
    *(datos + i) = reg;
  } 
  return 0;
}
```

Como se podra notar, la forma anterior es ineficiente por razones similares a las expuestas en la introducción.

### Forma 2

Declarar un arreglo dinamico (automatico) de un tamaño especificado en tiempo de ejecución. El siguiente codigo ilustra esta idea:

```C
#include <stdio.h>

int main() {
  float reg;
  int numReg;
  printf("Ingrese la cantidad de registros a leer: ");
  scanf("&d",numReg);
  float datos[numReg];
  for(int i = 0; i < numReg; i++) {
    printf("Dato[%d]: ", i);
    scanf("%f", &reg);    
    *(datos + i) = reg;
  } 
  return 0;
}
```

El problema de la forma anterior es que el compilador es previo a C99, no permite declarar un arreglo definiendo su tamaño a partir de una variable (siendo **numReg**) para el caso.

### Forma 3

Declarar un arreglo dinamico de un tamaño especificado en tiempo de ejecución empleando las funciones propias de la libreria estandar para tal caso (```malloc``` y ```calloc```). En lo que respecta al caso, esta es la mejor forma. La descripción y uso de estas funciones sera tratada en breve. Por ahora veamos el codigo asociado.

```C
#include <stdio.h>

int main() {
  float *datos;
  float reg;
  int numReg;
  printf("Ingrese la cantidad de registros a leer: ");
  scanf("&d",numReg);
  datos = (float *)malloc(numReg*sizeof(float));
  if (pF != NULL) {
    for(int i = 0; i < numReg; i++) {
      printf("Dato[%d]: ", i);
      scanf("%f", &reg);    
      *(datos + i) = reg;
  } 
  free(datos);
  return 0;
}
```

### Ejemplos

En los siguientes ejemplos se muestra la diferencia entre declarar una variable en el **stack** y declararla en el **heap** usando reserva dinamica de memoria.

1. **Creando una variable en el stack**
   
   ```{code-block} c
   #include <stdio.h>

   int main() {
     int a;
     int *p = &a;
     *p = 5;
     printf("*p = %d\n", *p);
     return 0;
   }
   ```
   
   El código anterior se puede simular a continuación:

   <iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=%23include%20%3Cstdio.h%3E%0A%0Aint%20main%28%29%20%7B%0A%20%20int%20a%3B%0A%20%20int%20*p%20%3D%20%26a%3B%0A%20%20*p%20%3D%205%3B%0A%20%20printf%28%22*p%20%3D%20%25d%5Cn%22,%20*p%29%3B%0A%20%20return%200%3B%0A%7D&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>
   
   La siguiente figura muestre el efecto del codigo anterior:

   ```{figure} ./local/img/CH_02-S05-fig6.png
   ---
   name: fig6
   ---
   Accediendo desde un apuntador a una variable definida en el **stack**.
   ```
   
   La siguiente figura muestre el efecto del codigo anterior:

   ![mem_heap](https://github.com/repos-SO-UdeA/laboratorios/blob/master/lab1/teoria/parte5/imagenes/malloc_help.png?raw=true)
   **Figura 4**. Asignacion dinamica de una variable usando malloc

2. **Creando una variable en el heap**
   
   ```{code-block} c
   #include <stdio.h>
   #include <stdlib.h>

   int main() {
     int *p;
     p = (int *)malloc(sizeof(int));
     if(p == 0) {
       printf("ERROR: Out of memory\n");
       return 1;
     }
     *p = 5;
     printf("*p = %d\n", *p);
     free(p);
     return 0;
   }
   ```

   A continuación, se puede simular el código anteriormente mostrado:

   <iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=%23include%20%3Cstdio.h%3E%0A%23include%20%3Cstdlib.h%3E%0A%0Aint%20main%28%29%20%7B%0A%20%20int%20*p%3B%0A%20%20p%20%3D%20%28int%20*%29malloc%28sizeof%28int%29%29%3B%0A%20%20if%28p%20%3D%3D%200%29%20%7B%0A%20%20%20%20printf%28%22ERROR%3A%20Out%20of%20memory%5Cn%22%29%3B%0A%20%20%20%20return%201%3B%0A%20%20%7D%0A%20%20*p%20%3D%205%3B%0A%20%20printf%28%22*p%20%3D%20%25d%5Cn%22,%20*p%29%3B%0A%20%20free%28p%29%3B%0A%20%20return%200%3B%0A%7D&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

   La siguiente figura muestre el efecto del codigo anterior:

   ```{figure} ./local/img/CH_02-S05-fig7.png
   ---
   name: fig7
   ---
   Asignacion dinamica de una variable en el **heap** usando `malloc`
   ```
   
## 4. Manejo dinamico de memoria mediante C

### 4.1. Operador `sizeof`

Este es un operador (no una funcion) que devielve la cantidad en bytes ocupada por una variable o algun tipo de dato. La sintaxis basica de este operador se muestra a continuacion:

```{code-block} c
sizeof ( type-name )  
```

**Ejemplo**

En el siguiente codigo ejemplo se muestran algunos ejemplos del uso de este operador:

```{code-block} c
#include <stdio.h>

int main() {
  short *p;
  long long a;
  printf("sizeof(short*) = %d\n",sizeof(short*));  // Pasando un tipo de dato
  printf("sizeof(p1) = %d\n",sizeof(p)); // Pasando una variable
  printf("sizeof(short) = %d\n",sizeof(short)); // Pasando un tipo de dato
  printf("sizeof(long long) = %d\n",sizeof(long long)); // Pasando un tipo de dato
  printf("sizeof(long long) = %d\n",sizeof(a)); // Pasando una variable
  printf("sizeof(double*) = %d\n",sizeof(double*)); // Pasando un tipo de dato
  printf("sizeof(char*) = %d\n",sizeof(char*)); // Pasando un tipo de dato
  printf("sizeof(char) = %d\n",sizeof(char)); // Pasando un tipo de dato
  return 0;
}
```

El código anterior puede ser simulado a continuación:

<iframe width="800" height="500" frameborder="0" src="https://pythontutor.com/iframe-embed.html#code=%23include%20%3Cstdio.h%3E%0A%0Aint%20main%28%29%20%7B%0A%20%20short%20*p%3B%0A%20%20long%20long%20a%3B%0A%20%20printf%28%22sizeof%28short*%29%20%3D%20%25d%5Cn%22,sizeof%28short*%29%29%3B%20%20%0A%20%20printf%28%22sizeof%28p1%29%20%3D%20%25d%5Cn%22,sizeof%28p%29%29%3B%20%0A%20%20printf%28%22sizeof%28short%29%20%3D%20%25d%5Cn%22,sizeof%28short%29%29%3B%20%0A%20%20printf%28%22sizeof%28long%20long%29%20%3D%20%25d%5Cn%22,sizeof%28long%20long%29%29%3B%20%0A%20%20printf%28%22sizeof%28long%20long%29%20%3D%20%25d%5Cn%22,sizeof%28a%29%29%3B%20%0A%20%20printf%28%22sizeof%28double*%29%20%3D%20%25d%5Cn%22,sizeof%28double*%29%29%3B%0A%20%20printf%28%22sizeof%28char*%29%20%3D%20%25d%5Cn%22,sizeof%28char*%29%29%3B%20%0A%20%20printf%28%22sizeof%28char%29%20%3D%20%25d%5Cn%22,sizeof%28char%29%29%3B%20%0A%20%20return%200%3B%0A%7D&codeDivHeight=400&codeDivWidth=350&cppShowMemAddrs=true&cumulative=false&curInstr=9&heapPrimitives=nevernest&origin=opt-frontend.js&py=c_gcc9.3.0&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

A continuacion se muestra la salida en el simulador para diferentes tipos de datos:


```{figure} ./local/img/CH_02-S05-fig8.png
   ---
   name: fig8
   ---
   Tamaño de diferentes tipos de datos.
   ```

### 4.2. Manejo dinamico de memoria

#### 4.2.1. Asignación de memoria: Memory Allocation o Malloc

La función ```malloc``` permite reservar un bloque de memoria (sin inicializar) en el heap. El prototipo de esta función se muestra a continuación:

```{admonition} malloc
:class: tip
**Sintaxis**: 

`void * malloc (size_t size)`


**Donde**:
* **`size`**: Numero de bytes que seran reservados

La funcion retorna la direccion en el heap a partir de la cual se reservó el tamaño de memoria solicitado o retorna **`NULL`** si no es posible reservar la cantidad de memoria. Cabe recordar que ```void*``` indica que la dirección retornada es genérica, es decir, en esa dirección se puede almacenar cualquier tipo de variable.
```

#### 4.2.2. Liberación de memoria: free

La función ```free``` libera un bloque de memoria previamente reservado en el heap. El prototipo de esta función se muestra a continuación:

```{admonition} free
:class: tip

**Sintaxis**:

`void free(void* pointer);`

**Donde**:
* **`pointer`**: apuntador con la dirección del bloque de memoria (previamente reservado con **`malloc`** o con **`calloc`**) a liberar. 
```

## x. Enlaces

* https://bytesoftheday.wordpress.com/2014/07/04/q14/
* https://www.cs.princeton.edu/courses/archive/spring20/cos217/
* https://embeddedwala.com/Blogs/embedded-c/memory-layout-of-c-program
* https://www.cs.mtsu.edu/~cs2170/C++labs/lab18/OSmemlayout.pdf
* https://d1b10bmlvqabco.cloudfront.net/attach/j6fe5friemd22w/hzd1madqsie3ts/j7kw6i4tmqf8/61C_Note_1_Memory.pdf
* https://ocw.mit.edu/courses/6-s096-introduction-to-c-and-c-january-iap-2013/bba9056d5290198d563edc47dfcff0e9_MIT6_S096_IAP13_lec3.pdf
* https://cs61c.org/su24/
* http://wla.berkeley.edu/~cs61c/fa17/
* https://www.cs.princeton.edu/courses/archive/fall07/cos217/index.html
* https://web2.qatar.cmu.edu/~mhhammou/15122-s23/lectures/21-cmem/writeup/pdf/main.pdf
* https://cs.gmu.edu/~zduric/cs262/Slides/teoX.pdf
* https://d1b10bmlvqabco.cloudfront.net/attach/j6fe5friemd22w/hzd1madqsie3ts/j7kw6i4tmqf8/61C_Note_1_Memory.pdf
* https://www.cs.princeton.edu/courses/archive/fall07/cos217/
* https://www.cs.mtsu.edu/~cs2170/C++labs/lab18/OSmemlayout.pdf
* https://web2.qatar.cmu.edu/~mhhammou/15122-s23/lectures/21-cmem/writeup/pdf/main.pdf
* https://www.cs.princeton.edu/courses/archive/spr24/cos126/schedule/
* https://github.com/vishwa27yvs/Intro-to-Computer-Science-COS-126